# Clasificación de imágenes con redes profundas

*Por Ernesto Ignacio Borbón Martínez, Luis Felipe Villaseñor Navarrete, José de Jesús Gutiérrez Aldrete y Gerardo Villegas Contreras*

Es la hora de la verdad. Anteriormente habíamos utilizado métodos clásicos y modelos perceptrón multicapa para clasificar imágenes usando como entrada a los pixeles de las mismas y a unos descriptores. Hubo modelos satisfactorios y otros lejos de ser exitosos. En esta ocasión continuando con este caminar de la clasificación de imágenes, evaluaremos modelos basados en redes neuronales convolucionales.

Las imágenes que utilizaremos serán unas antiguas conocidas: el conjunto de datos Fashion, el conjunto Emojis y el conjunto Navigability.

## Las bases de datos

Vamos a recordar en qué consistían los conjuntos de datos:

El primero es el conjunto de datos Fashion-MNIST, el cual consiste de 70,000 imágenes (60,000 de entrenamiento y 10,000 de prueba) de 10 diferentes prendas de vestir en escala de grises con un tamaño de 28x28, acompañado de sus etiquetas. Sus clases son:

* 0 T-shirt/top-Playera 
* 1 Trouser-Pantalón
* 2 Pullover-Suéter
* 3 Dress-Vestido
* 4 Coat-Abrigo
* 5 Sandal-Sandalia
* 6 Shirt-Camisa
* 7 Sneaker-Tenis
* 8 Bag-Bolsos
* 9 Ankle boot-Calzado alto

El segundo conjunto de datos consiste en 2470 imágenes binarias de emojis dibujados a mano con un tamaño de 32x32. Los diferentes emojis en la base de datos son caritas enojadas (clase 1) felices (2), tristes (4), sorprendidas (5) y el emoji de popó (clase 3). 

El tercer y último conjunto de datos, que nombramos Navigability o Navegabilidad, consta de 2000 secciones de imágenes de mapas de elevación de la superficie de Marte tomadas del sitio de HiRISE. Cada registro en el conjunto de datos contiene cuatro imágenes de un área de 200m x 200m acompañados de un indicador de qué tan navegable es esa zona en una escala del 1 al 4 (1: Muy poco navegable – 4: Muy navegable). La primera imagen contiene las alturas, la segunda es un gradiente, la tercera indica las elevaciones en el área y la cuarta las depresiones.

Puedes encontrar el código de este blog junto con las bases de datos emojis y navigability en https://github.com/ShoyChoy/clasificacion-profunda.git


## Procedimientos preliminares

Importación de librerías.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pickle

from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, recall_score, mean_squared_error

import tensorflow as tf
from tensorflow.keras import layers, models

from tqdm.notebook import trange

Carga de datos.

In [ ]:
fashion_mnist = keras.datasets.fashion_mnist
(fashion_tr_im,fashion_tr_lab),(fashion_test_im,fashion_test_lab) = fashion_mnist.load_data()

In [ ]:
emojis = np.loadtxt("emojis.txt") 
x_emojis = emojis[:,1:]
y_emojis = emojis[:,0]
y_emojis -= 1

In [ ]:
inputFile = open('navigability.obj', 'rb')
nav = pickle.load(inputFile)

Para poder utilizar estos datos en los modelos de a continuación, es necesario realizar unos ajustes. En primer lugar pasar los vectores del conjunto Emojis a matrices de 32x32; y en segundo lugar dividir los datos del conjunto Navigability en las entradas y las salidas.

In [ ]:
reshaped_x_emo = []
for i in range(x_emojis.shape[0]):
  reshaped_x_emo.append(x_emojis[i,:].reshape(32,32))
x_emojis = np.array(reshaped_x_emo)

In [ ]:
x_nav = []
y_nav = []
for i in trange(len(nav)):
  x_nav.append(nav[i][2:])
  y_nav.append(nav[i][0])
x_nav = np.array(x_nav)
y_nav = np.array(y_nav)

  0%|          | 0/849 [00:00<?, ?it/s]

In [ ]:
x_nav = x_nav.reshape(849,20,20,4)

## Evaluación de redes neuronales convolucionales

Las redes neuronales creadas para los datos de Fashion y Emojis tienen 2 capas de convolución (con kernel de 3x3) seguidas de 2 capas de Max-Pooling (tamaño de 2x2) y una última capa de convolución de la cual continúa el flatten para poder ser alimentados a las capas “fully-conected” donde la primera tiene una función de activación relu y la última es la de salida. 

Para los datos de Fashion la primera capa tiene una entrada (28, 28, 1) donde 28 es el tamaño de las imágenes y 1 es la dimensión. Mientras que para emojis usa una entrada de (32, 32, 1). La red para los datos de Navigability tiene una entrada de (20, 20, 4) donde 20x20 es el tamaño de las imágenes y 4 es la dimensión por las 4 imágenes que tiene cada registro, es muy similar a las anteriores, pero a diferencia de ellas su capa de salida tiene una función de activación lineal. Esto porque, como recordarás, este problema puede tratarse como uno de regresión.

De esta manera, los modelos para Fashion y Emojis se evalúan con la exactitud y la sensbilidad/recall por clase, mientras que el modelo para Navigability se evalúa con el error cuadrado medio.

### Fashion

In [ ]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))

In [ ]:
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10))

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
model.fit(fashion_tr_im,fashion_tr_lab, epochs=15)

In [ ]:
y_pred = model.predict(fashion_test_im)
y_pred = np.argmax(y_pred, axis=-1)

In [ ]:
acc = accuracy_score(fashion_test_lab, y_pred)
rec = recall_score(fashion_test_lab, y_pred, average = None)

In [ ]:
print('Accuracy: ', acc)
print('Recall: ', rec)

Accuracy:  0.8951
Recall:  [0.847 0.979 0.852 0.925 0.786 0.986 0.686 0.963 0.982 0.945]


### Emojis

In [ ]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))

In [ ]:
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(5))

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

folds = 5
kf = KFold(n_splits= folds, shuffle= True)

In [ ]:
accs = []
recs = []
for train_index, test_index in kf.split(x_emojis):
  model.fit(x_emojis[train_index], y_emojis[train_index], epochs=15)
  y_pred_emoji = model.predict(x_emojis[test_index])
  y_pred_emoji = np.argmax(y_pred_emoji, axis=-1)
  accs.append(accuracy_score(y_emojis[test_index], y_pred_emoji))
  recs.append(recall_score(y_emojis[test_index], y_pred_emoji, average = None))

In [ ]:
np.mean(accs)

0.9894736842105264

In [ ]:
recsA = np.mean(recs, axis = 0)
recsA

array([0.9742803 , 0.9962963 , 1.        , 0.98523322, 0.99191919])

### Navegabilidad

In [ ]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(20, 20, 4)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))

In [ ]:
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(1, activation='linear'))

In [ ]:
model.compile(optimizer='adam',
              loss='mse',
              metrics='mse')

folds = 5
kf = KFold(n_splits= folds, shuffle= True)

In [ ]:
mse = []
for train_index, test_index in kf.split(x_nav):
  model.fit(x_nav[train_index], y_nav[train_index], epochs=30, verbose=False)
  y_pred = model.predict(x_nav[test_index])
  mse.append(mean_squared_error(y_nav[test_index], y_pred))

In [ ]:
print('MSE: ', np.mean(mse))

MSE:  0.3367238746278761


## Resultados

Se obtuvieron resultados sobresalientes tanto en los problemas de clasificación como en el de regresión. Con el conjunto de datos Fashion, se obtuvo una exactitud del 90%, alcanzando los mayores recalls en las clases 0, 1, 3, 5, 7, 8 y 9 (T-shirts/top, pantalones, vestidos, sandalias, tenis y zapatos altos) con valores entre 92% y 99%, mientras que el peor recall por mucho lo obtuvo la clase 6 (camisas) con un valor de 67%.

Por otro lado, con el conjunto de datos de los Emojis se obtuvieron aún mejores resultados. La exactitud fue del 99% y todos los recalls fueron mayores al 97%. El peor recall fue el de la clase 1 (caritas enojadas) con 97% y el mejor fue el de la clase 3 (emoji de popó) con un sorprendente 100%.

Finalmente, el modelo de regresión con el conjunto de datos de las imágenes de Marte obtuvo un error cuadrado medio de 0.34, el cual es satisfactorio.
